# [미션] "인공지능", "데이터" 연관 채용공고 크롤링하기
## 참가팀 소개
* 팀명: 취업엔딩🌸
* 팀원: 이시현, 유진아, 김하경, 김윤민, 김나리
* 내용 소개: <br>
점핏 홈페이지를 통해 '빅데이터 엔지니어', '인공지능/머신러닝' 포지션을 기준으로 <br>
특정 연차 경력의 채용 공고를 스크랩핑해오며, (신입, 1년차, 2년차 등 각각) <br>
추가적으로 해당 채용 공고의 회사 평점 정보를<br>
잡플래닛 사이트에서 따로 가져와서 하나의 엑셀파일로 만드는 코드를 작성하였다.<br>

* 사용 데이터:
    - 점핏 : 회사명, 공고명, 기술스택, 주요업무 , 자격요건, 지원마감일 등..
    - 잡플래닛 : 회사 평점
    
* 스크래핑 조건:
    -  '빅데이터 엔지니어', '인공지능/머신러닝' 포지션만
    -  경력 조건을 함수의 input 데이터로 하여 신입, 1년, 2년 등 엑셀파일마다 단일하게 가져올 수 있게함.
    - 결과 파일은 신입, 1년차 채용 공고 스크래핑 파일을 첨부함.
* 스크래핑 방법:
    - 셀레니움을 통해 가상 웹브라우저로 접근하는 방법을 사용함.

In [2]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
from datetime import datetime
import time
import re

In [3]:
from webdriver_manager.chrome import ChromeDriverManager # 자동으로 chrome driver 다운로드 용 
from selenium.webdriver.chrome.service import Service # 다운로드된 크롬드라이버 파일을 연결하기 위해 활용

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import warnings
warnings.filterwarnings("ignore") # 불필요한 Warning 메시지를 꺼줍니다.

### 1. 크롬 가상 드라이브 세팅

In [4]:
service = Service(executable_path = ChromeDriverManager().install())

driver = webdriver.Chrome(service=service)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/hk/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


### 2. 잡플래닛 스크래핑 코드 함수

In [5]:
# 잡플래닛 스크래핑 함수
def job_planet_scraping(company):
    url = "https://www.jobplanet.co.kr/search?query=" + company
    
    # 사이트 차단 대비 유저 정보
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36'}

    res = requests.get(url, headers=headers).content
    soup = BeautifulSoup(res,'html.parser')
    
    # 크롤링 에러날 경우 '정보없음'으로 처리
    rate = "정보없음"
    
    try:
        # 평점 긁어오기
        star = soup.find('span',{'class':'rate_ty02'})
        # 평점 숫자만 나오도록 수정
        rate = star.get_text()

    except:
        pass
    
    return rate

### 3. 전체 스크래핑 코드 함수 

In [6]:
def scraping(year):    
    # 빅데이터 엔지니어, 인공지능/머신러닝, 신입 
    df = pd.DataFrame({'jobpost_url':[],
                  'skill_stack':[]})
    
    url = "https://www.jumpit.co.kr/positions?jobCategory=8&career={}".format(year)
    print(df)
    
    driver.get(url)
    print(driver.current_url)

    # 계속 스크롤 다운하면서 데이터를 다 조회할 때
    SCROLL_PAUSE_SEC = 1

    # 스크롤 높이 가져옴
    last_height = driver.execute_script("return document.body.scrollHeight")
    print(last_height)

    while True:
        # 끝까지 스크롤 다운 
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(SCROLL_PAUSE_SEC)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        print(new_height, last_height)
        if new_height == last_height:
            time.sleep(SCROLL_PAUSE_SEC)
            break # while 문을 나가서 last_height 업데이트 
        last_height = new_height
    # 스크롤 한 뒤 다시 web html 가져오기 
    web = BeautifulSoup(driver.page_source, 'html.parser')

    # 전체 공고 내용 
    # positions = web.find_all('div', {'class':'sc-dJjYzT jtdhQb'})
    positions = web.find_all('div', {'class':'sc-dlVxhl fUeray'})

    #company_name = []
    #jobposting_name = []
    jobposting_url = []
    skill_stack = []

    for position in positions:
        url = position.find('a', {'target':'_self'}).attrs['href'] # 공고 url 
        #skill = position.find('ul', {'class':'sc-fKVqWL jcccsQ'}).get_text().split()
        skill = position.find('ul', {'class':'sc-iwjdpV keRuBz'}).get_text().split()

        tmp = {'jobpost_url': url, 'skill_stack': skill}

        df = df.append(tmp, ignore_index = True)
        #position.find('img').attrs['alt'] # 회사명 
        #position.find('h2', {'class':'position_card_info_title'}).get_text() # 공고명
    print("url 수집 완료")
    print(df)
    
    df['jobpost_name'] = ['0']*len(df) # 공고명 
    df['company_name'] = ['0']*len(df) # 기업명 
    
    # 여기에 추가 -잡플래닛
    df['company_star'] = ['0']*len(df) # 기업 평점
    
    df['task'] = ['0']*len(df) # 주요업무
    df['qualification'] = ['0']*len(df) # 자격요건
    df['treatment'] = ['0']*len(df) # 우대사항
    df['welfare'] = ['0']*len(df) # 복지 및 혜택
    df['end_day'] = ['0']*len(df) # 마감일
    df['education'] = ['0']*len(df) # 학력 

    #전체 공고별 데이터 불러오기 
    position_url = 'https://www.jumpit.co.kr'

    for i in range(len(df)):
        # 공고 각각에 접근 
        new_url = position_url + df.at[i, 'jobpost_url']
        print(new_url)

        driver.get(new_url)
        time.sleep(1)
        new_web = BeautifulSoup(driver.page_source, 'html.parser')

        # 공고명 
        jobpost_name = new_web.find('h1').get_text()
        #print(jobpost_name)
        df.at[i, 'jobpost_name'] = jobpost_name

        # 기업명 
        #comp_name = new_web.find('div', {'class':'styles_position_title_box_desc__uvEoB'}).get_text()
        comp_name = new_web.find('div', {'class':'position_title_box_desc'}).get_text()
        #print(comp_name)
        df.at[i, 'company_name'] = comp_name
        
        # 여기에 추가 -잡플래닛
        star = job_planet_scraping(comp_name)
        df.at[i, 'company_star'] = star


        # 주요업무
        #task = new_web.find_all('dl', {'class':'styles_root__FK3fp'})[1].pre.get_text()
        task = new_web.find_all('dl', {'class':'sc-gWXbKe hvgMyA'})[1].pre.get_text()
        task = task.replace('\n', '')
        df.at[i, 'task'] = task

        # 자격요건
        #qualification =new_web.find_all('dl', {'class':'styles_root__FK3fp'})[2].pre.get_text()
        qualification =new_web.find_all('dl', {'class':'sc-gWXbKe hvgMyA'})[2].pre.get_text()
        qualification = qualification.replace('\n', '')
        df.at[i, 'qualification'] = qualification

        # 우대사항
        #treat =new_web.find_all('dl', {'class':'styles_root__FK3fp'})[3].pre.get_text()
        treat =new_web.find_all('dl', {'class':'sc-gWXbKe hvgMyA'})[3].pre.get_text()
        treat = treat.replace('\n','')
        df.at[i, 'treatment'] = treat

        # 복지 및 혜택
        #wel =new_web.find_all('dl', {'class':'styles_root__FK3fp'})[4].pre.get_text()
        wel =new_web.find_all('dl', {'class':'sc-gWXbKe hvgMyA'})[4].pre.get_text()
        wel = wel.replace('\n','')
        df.at[i, 'welfare'] = wel

        # 학력 
        edu = new_web.find_all('dl', {"class":"styles_item__LhiHT"})[1].dd.get_text() 
        df.at[i, 'education'] = edu

        # 마감일
        end_day = new_web.find_all('dl', {"class":"styles_item__LhiHT"})[2].dd.get_text()
        df.at[i, 'end_day'] = end_day
        
        # 경력
        if year == 0 : 
            year = '신입'
        df.at[i, 'career'] = int(year)
    return df

year == 0 (신입 경력) 일 때의 채용 공고 스크래핑하여 저장.

In [7]:
df_0 = scraping(0)

Empty DataFrame
Columns: [jobpost_url, skill_stack]
Index: []
https://www.jumpit.co.kr/positions?jobCategory=8&career=0
1908
3197 1908
4539 3197
5858 4539
6529 5858
6529 6529
url 수집 완료
       jobpost_url                                        skill_stack
0   /position/8084          [sw·, machinelearning·, ai/인공지능·, python]
1   /position/6132  [c#·, python·, pytorch·, tensorflow·, keras·, ...
2   /position/8022     [python·, pytorch·, tensorflow·, deeplearning]
3   /position/7255  [python·, ai/인공지능·, machinelearning·, r·, aws·...
4   /position/5411  [tensorflow·, pytorch·, linux·, python·, ai/인공지능]
..             ...                                                ...
66  /position/3637                       [python·, deeplearning·, sw]
67  /position/2763  [deeplearning·, tensorflow·, c++·, java·, pyth...
68  /position/2405              [c++·, python·, tensorflow·, pytorch]
69  /position/1761  [c++·, c#·, opencv·, communication·, visual, s...
70  /position/1322               [machinelear

In [8]:
df_0.to_csv("career_0_result.csv", index = False, encoding="utf-8-sig")

In [9]:
df_0.head()

,jobpost_url,skill_stack,jobpost_name,company_name,company_star,task,qualification,treatment,welfare,end_day,education,career
0,/position/8084,"[sw·, machinelearning·, ai/인공지능·, python]",질병치료 관련 개발 연구원(신입~3년),넷타겟,0.0,"• 모집분야: Systems Biology, Computational Biology...",• 석사 이상• 신입~3년 경력,• 제약회사/바이오텍 근무 경력 선호\t\t\t\t\t\t\t\t\t• 시스템생물학...,"• 개인 장비 지원: 최고급 PC 및 노트북, 구글클라우드 및 자체서버 지원, 사무...",2022-05-03,석사졸업 이상,신입
1,/position/6132,"[c#·, python·, pytorch·, tensorflow·, keras·, ...",딥러닝 기반 이미지 프로세싱 개발자,에타맥스,2.9,"• 딥러닝을 활용한 반도체, LED 소자 결함 검출 연구 개발",• 관련 전공 석사 이상,"• 대용량 이미지 처리 프로젝트 경험자• Object Detection, Image...","• 주 1회 재택근무 가능• 주 5일 유연 근무제 운용, 눈치보이지 않는 자유로운 ...",2022-05-07,석사졸업 이상,신입
2,/position/8022,"[python·, pytorch·, tensorflow·, deeplearning]",[전문연구요원] 생성모델 연구 및 제품 개발자 모집,솔트룩스,2.8,• 생성 모델 연구 및 제품 개발 - GAN 응용 연구(입모양 및 포즈 생성) ...,• 학력 : 석사학위 이상• 경력 : 신입~경력 3년• 기술스택 : python(p...,• 관련 전공 이수 • Openpose/Mmpose를 활용한 연구 및 개발 • SO...,• 임직원 및 가족 의료비 및 건강검진 지원• 5성급 호텔 숙박료 지원• 회식비 및...,상시,석사졸업 이상(졸업예정자 가능),신입
3,/position/7255,"[python·, ai/인공지능·, machinelearning·, r·, aws·...",개발 역량이 있는 AI/Data Engineer,쿤텍,4.1,"• AI / Data 컨설팅 - 기업 대상 DataOps, MLOps 컨설팅 - 머...","• 컴퓨터공학, 데이터, AI, 수학/통계학 등 전산계열 전공자• 개발 경험",• 통계적 기법을 통한 데이터 분석 역량• R/Python 프로그래밍 및 기본적인 ...,• 쿤텍은 자유로운 분위기 안에서 본인이 맡은 역할과 책임을 다하는 업무 환경을 지...,상시,대학교졸업(4년) 이상,신입
4,/position/5411,"[tensorflow·, pytorch·, linux·, python·, ai/인공지능]",[석사이상] AI Engineer,라온데이터,5.0,• Natural Language / Vision / Audio 등 다양한 도메인의...,• 인공지능 관련 분야 석/박사 학위 [필수]• 수습기간 : 3개월• 기본적인 Li...,• 학원 / 튜터링 프로그램을 제외한 최신 인공지능 기법을 활용한 서비스 / 프로젝...,• 코어근무제 - 11:30분 출근 / 자율 퇴근• 모든 팀원간의 수평적인 관계• ...,2022-04-29,석사졸업 이상(졸업예정자 가능),신입


year == 1(1년차 경력) 일 때의 채용 공고 스크래핑하여 저장.

In [ ]:
df_1 = scraping(1)
df_1.to_csv("career_1_result.csv", index = False, encoding="utf-8-sig")
df_1.head()

### 5. 크롬 가상 브라우저 닫기 

In [10]:
driver.close()
driver.quit()